In [84]:
import requests
import pickle
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

In [85]:
def skip_map():
    global wait
    try:
        ymap = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div[4]/div[1]/ymaps/ymaps/ymaps/ymaps[1]')
        button_resize = ymap.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[4]/div[1]/ymaps/ymaps/ymaps/ymaps[4]/ymaps[3]/ymaps/ymaps/ymaps[1]/ymaps')
        time.sleep(5)
        button_resize.click()
        button_resize.click()
        
        time.sleep(5)
        element_deal = wait.until(EC.element_to_be_clickable(driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[2]/div[2]/div/div/div[1]/div[2]/div/div/div/div[1]/div[1]/a')))
        element_deal.click()
    except:
        pass

In [89]:
def parse_not_found_article():
    time.sleep(2)
    global my_dict, item, driver
    try:
        article_dict = dict()
        group_dict = dict()
        
        not_found_article = driver.find_element(By.CLASS_NAME, "sc-bb20ea4e-0")
        data_testid_makersTable = not_found_article.find_elements(By.CLASS_NAME, 'sc-c9199ccb-3')
        title = not_found_article.find_element(By.CLASS_NAME, 'sc-ce518372-0').text
        print(len(data_testid_makersTable))
        for helpfull_data in data_testid_makersTable:
            
            factory_plus_name = []
            
            articles = helpfull_data.find_elements(By.CLASS_NAME, 'sc-c9199ccb-7')
            factorys = helpfull_data.find_elements(By.CLASS_NAME, 'sc-c9199ccb-5')
            if (factorys[0].text != " ") & (factorys[0].text != None) & (len(factorys[0].text) > 2):
                factory = factorys[0].text
                article = articles[0].text
            else:
                factory = factorys[1].text
                article = articles[1].text
            
            try:
                name = helpfull_data.find_element(By.CLASS_NAME, 'sc-c9199ccb-8').text
            except:
                name = "-"
                
            article = article.replace(" ", "")
            factory_plus_name.append(factory)
            factory_plus_name.append(name)
            
            print(f'{article} -- {factory} -- {name}')
            
            if article in article_dict:
                continue
            else:
                article_dict[article] = factory_plus_name
                
        group_dict[title] = article_dict
        my_dict[item] = group_dict
    except Exception as e:
        # print(e)
        pass

In [87]:
def parse_found_articles():
    time.sleep(2)
    global my_dict, item, driver
    try:
        direct_relationship = []
        group_dict = dict()
        
        block_with_all_articles = driver.find_element(By.CLASS_NAME, "sc-661d0768-0")
        groups_of_articles = block_with_all_articles.find_elements(By.CLASS_NAME, 'sc-6a057c55-0')
        for group in groups_of_articles:
            article_dict = dict()
            title = group.find_element(By.TAG_NAME, 'h2').text
            group_of_articles_by_title = group.find_elements(By.CLASS_NAME, 'sc-b28de792-0')
            
            for helpfull_data in group_of_articles_by_title:
                factory_plus_name = []
                factory = helpfull_data.find_element(By.CLASS_NAME, 'sc-9d72349a-7').text
                article = helpfull_data.find_element(By.CLASS_NAME, 'sc-9d72349a-5').text
                try:
                    name = helpfull_data.find_element(By.CLASS_NAME, 'sc-9d72349a-4').text
                except:
                    name = "-"
                article = article.replace(" ","")
                factory_plus_name.append(factory)
                factory_plus_name.append(name)
                
                print(f'{article} -- {factory} -- {name}')
                if article in article_dict:
                    continue
                else:
                    article_dict[article] = factory_plus_name
                    
            group_dict[title] = article_dict
            
        my_dict[item] = group_dict
    except:
        # print("parse_found_articles broke")
        pass

In [92]:
my_dict = dict()
file_path_txt = 'output.txt'
art = "3C9885405GMVQK"

chrome_options = uc.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = uc.Chrome(options=chrome_options)
driver.get("https://emex.ru")

wait = WebDriverWait(driver, 10)
# element = wait.until(EC.element_to_be_clickable((By.ID,'someid')))

with open(file_path_txt, 'r') as file:
    data = file.readlines()
    
try:
    for item in data:
        time.sleep(2)
        
        item = item.replace("\n","")
        input_element = driver.find_element(By.CLASS_NAME, 'e-inputField')
        input_element.clear()
        input_element.send_keys(item)
        time.sleep(3)
        
        actions = ActionChains(driver) 
        actions.send_keys(Keys.ENTER)
        actions.perform()
        time.sleep(3)
        
        skip_map()
        try:
            parse_not_found_article()
            
            parse_found_articles()

            with open('Emex_data.pkl', 'wb') as file:       
                pickle.dump(my_dict, file)
                
        except (NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException) as e:
            
            print(item + ": no same arts in system")
            continue    

except Exception as e:
    
    print(f"An error occurred during the request: {str(e)}")

3
3C1864777 -- VAG -- -
3C1864777A4J4 -- VAG -- НАКЛАДКА/ABDECKUNG
3W1864777A -- VAG -- -
1K0201557A -- VAG -- УПЛОТНИТЕЛЬНОЕ КОЛЬЦО
3C0201557 -- VAG -- Уплотнит.
1K0201557 -- VAG -- Манжетное уплотнение
DEBZ0619 -- DePPuL -- Кольцо уплотнительное крышки бензобака
HLKLC230 -- HLLT -- Кольцо уплотнительное крышки бензобака
WEKLC035 -- WenderW -- Кольцо уплотнительное крышки бензобака
ZVBZ0619 -- ZZVF -- Кольцо уплотнительное крышки бензобака
5N09552059B9 -- VAG -- КОЛПАЧОК Ч РНЫЙ SATINSCHWARZ
PWAC006 -- Patron -- Колпачок стеклоочистителя
6RU8075229B9 -- VAG -- МОЛДИНГ ЗАДНЕГО БАМПЕРА ПРАВЫЙ
6Q0864732 -- VAG -- ПЕННЫЙ НАПОЛНИТЕЛЬ
1K0919341F9B9 -- VAG -- ВТУЛКА С КРЫШКОЙ / 1K0919341F9B9
N90544501 -- VAG -- Кольцо упл. пробки радиатора
A005K147 -- Akrado -- УПЛОТНИТЕЛЬНОЕ КОЛЬЦО VAG N90544501
AL300519 -- AllRing -- Кольцо уплотнительное
ORINGKIT3 -- Auto-GUR -- НАБОР УПЛОТНИТЕЛЬНЫХ КОЛЕЦ O-RING (404 ШТУК)
ORINGKIT4 -- Auto-GUR -- НАБОР УПЛОТНИТЕЛЬНЫХ КОЛЕЦ O-RING (382 ШТУК)
ORINGKIT5 -- A

In [91]:
# for key, value in my_dict.items():
#     print(f"{key}: {value}")  

my_dict

{'3C1864777A': {'Товары с похожими номерами': {'3C1864777': ['VAG', '-'],
   '3C1864777A4J4': ['VAG', 'НАКЛАДКА/ABDECKUNG'],
   '3W1864777A': ['VAG', '-']}},
 '3C0857351E': {'Товары с похожими номерами': {'3C0857351E3F4': ['VAG',
    'Крышка пеп'],
   '3C0857351E3H5': ['VAG', 'Крышка пеп'],
   '3C0857351EZ88': ['VAG', 'Крышка пеп'],
   '3C0857351EZQV': ['VAG', 'Крышка пепельницы'],
   '3C0857351EZQW': ['VAG', 'Крышка пеп'],
   '3C0857351EZQX': ['VAG', 'Крышка пепельницы']}}}

In [ ]:
sc-bb20ea4e-0 eJwrFq ----> Товары с похожими номерами
    # data-testid="Makers:block:makersTable"   
        sc-c9199ccb-3 kgwsBQ e-link e-link--themeDefault
            sc-c9199ccb-4 kBrdDp
                sc-c9199ccb-5 eBCQKM ---> Производитель
                sc-c9199ccb-7 ewlDVi ----> Артикул
            sc-c9199ccb-6 kjyODf
                sc-c9199ccb-8 fPtOcN ----> NAME


sc-661d0768-0 SqIZI sc-85373483-2 dzvwJo ---> блок со всем

    sc-6a057c55-0 fjCCsj
        sc-b28de792-0 HJime
            sc-9d72349a-2 clQBXN
                <span> sc-9d72349a-3 hZJNAB <span>
                    sc-9d72349a-7 bsrna-d e-link e-link--themeDefault ----> Производитель
                    sc-9d72349a-5 jkObUb e-link e-link--themeSolid ----> Артикул
                <span> sc-9d72349a-4 dfYmOZ <span> ----> NAME

    sc-6a057c55-0 fjCCsj
        sc-b28de792-0 HJime
            sc-9d72349a-2 clQBXN
                <span> sc-9d72349a-3 hZJNAB <span>
                    sc-9d72349a-7 bsrna-d e-link e-link--themeDefault ----> Производитель
                    sc-9d72349a-5 jkObUb e-link e-link--themeSolid ----> Артикул
                <span> sc-9d72349a-4 dfYmOZ <span> ----> NAME

list("Offers:block:tableoriginals", "Offers:test:replacementsTableTitle", "Offers:test:originalsTableTitle")

In [3]:
file_path_txt = 'output.txt'

with open(file_path_txt, 'r') as file:
    data = file.readlines()
    
translator = str.maketrans("", "", "\n")    

for item in data:
    item = item.translate(translator)
    print(item)

1K0201557A
5N09552059B9
6RU8075229B9
6Q0864732
1K0919341F9B9
N90544501
03C121118D
N0195307
1K0880201CA1QB
1K2820103B
3C1867012DNTCJ
6RU867811A
6R4837879
5M0881479E7G8
022103139B
3C2867179B7R3
6Q0803127C
1S0959802
3C0857351E
3C1864777A
N90344501
6C0122447S
3C8858366S46X
3C0881314C31X
3C2857508BL9B9
036105269DX
6RU809605A
3C9861691
6RU881318B
6RU945106A
030103037E
6R0857552AM4T6
3C0863330A7G8
3C088225230T
N91004001
N90808201
6RU867427B
3C5885405HDRCW
3C0853651BPPWF
6C0819448
6RU885901FLK6
D181220A1
4A0937530
3C1857004AJTKV
N90987602
6Q0820129
6C0419685A
B000750M7
3C2867172B21X
3C9885405GMVQK
6C0853688B
N10247504
3C2868157A95T
3C4839016A
3C1823755C
6RU837085B
3C0853993A3FZ
6R0412341A
N90508801
3C5863470
6X0867549
09G301951A
6RF886683AQ70
3C0857511DTLY
6RF941531D
3C088148031X
6RU413031B
6RU867212F
N91027601
3C0864521H68
3C0880241C
3C2857114D1QB
5M0881480E30T
3C8831412A
3C0837216EINB
3C0881806MMXRS
WHT006990A
6C0122101AP
3C0864211B70C
8D0973822
3C1857212AD3F4
3C1867012DPULC
6RF819704CN0R
3C

In [86]:
base_url = "https://emex.ru/products/"
file_path_txt = 'output.txt'

translator = str.maketrans("", "", "\n")

driver = uc.Chrome(chrome_options=chrome_options)

with open(file_path_txt, 'r') as file:
    data = file.readlines()
    
for item in data:   
    
    item = item.translate(translator)
    # url = base_url + str(item)
    # print(url)
    # driver.get(url)
    # time.sleep(3)
    try:

        same_art_elements = driver.find_elements(By.CLASS_NAME, 'sc-f2a13e83-0 jVNWio')
        print("same_art_elements")

        for same_art_element in same_art_elements:

            detale_elements = same_art_element.find_elements(By.CLASS_NAME, 'sc-6e4a9aa6-3 sBrbz')
            print("detale_elements")

            for element in detale_elements:

                time.sleep(3)
                link_elements = element.find_elements(By.TAG_NAME, 'a')
                print("link_elements")
                list_items = []

                for text in link_elements:
                    print("text")
                    list_items.append(text)

                my_dict[item] = list_items

    except (NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException) as e:

        print(item + ": no same arts in system")
        continue    
        
    driver.close()

https://emex.ru/products/6Q0820129
https://emex.ru/products/6C0419685A
https://emex.ru/products/B000750M7
https://emex.ru/products/3C2867172B21X
https://emex.ru/products/3C9885405GMVQK
https://emex.ru/products/6C0853688B
https://emex.ru/products/809-075
https://emex.ru/products/N10247504


KeyboardInterrupt: 